In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GenerationConfig, AutoTokenizer, GPT2Config
import torch
import torch.nn as nn
import os
from pathlib import Path

import transformers
from onnxruntime import InferenceSession
from transformers.onnx.features import FeaturesManager
from optimum.exporters.tasks import TasksManager

In [9]:
import numpy as np

def save_to(x, fp):
    print(x.dtype)
    np_x = x.detach().numpy()
    with open(fp, 'wb') as f:
        np_x.tofile(f)

In [10]:
torch.manual_seed(2333)
torch.set_num_threads(1)

In [4]:
gpt2_features = list(FeaturesManager.get_supported_features_for_model_type("gpt2").keys())
gpt2_features

['default',
 'default-with-past',
 'causal-lm',
 'causal-lm-with-past',
 'sequence-classification',
 'token-classification']

In [12]:
gpt2_tasks = list(TasksManager.get_supported_tasks_for_model_type("gpt2", "onnx").keys())
gpt2_tasks

['default',
 'default-with-past',
 'causal-lm',
 'causal-lm-with-past',
 'sequence-classification',
 'token-classification']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('model_onnx')
session = InferenceSession("model_onnx/model.onnx")
config = GPT2Config()
onnx_config = 

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.return_dict = False
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

In [7]:
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature='default-with-past')
onnx_config = model_onnx_config(model.config)

In [12]:
onnx_inputs, onnx_outputs = transformers.onnx.export(
    preprocessor=tokenizer,
    model=model,
    config=onnx_config,
    opset=13,
    output=Path("gpt2_model.onnx")
)

/users/Zijian/.local/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py:794: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:


In [14]:
print(onnx_inputs)
print(onnx_outputs)

['input_ids', 'past_key_values', 'attention_mask']
['last_hidden_state', 'present.0.key', 'present.0.value', 'present.1.key', 'present.1.value', 'present.2.key', 'present.2.value', 'present.3.key', 'present.3.value', 'present.4.key', 'present.4.value', 'present.5.key', 'present.5.value', 'present.6.key', 'present.6.value', 'present.7.key', 'present.7.value', 'present.8.key', 'present.8.value', 'present.9.key', 'present.9.value', 'present.10.key', 'present.10.value', 'present.11.key', 'present.11.value']


In [19]:
gen_cfg = GenerationConfig(
    pad_token_id = tokenizer.eos_token_id, 
    use_cache = True,
    max_new_tokens = 128)

In [20]:
%%time
output = model.generate(**encoded_input, generation_config=gen_cfg)
print(tokenizer.decode(output[0]))

Replace me by any text you'd like.

I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you
CPU times: user 5.14 s, sys: 264 ms, total: 5.41 s
Wall time: 5.46 s


In [5]:
input_names = ["text"]
output_names = ["prediction"]
dummy_input = torch.randint(0, 30522, (64, 20))

torch.onnx.export(bert,
                  dummy_input,
                  "./bert.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [10]:
save_to('.', x, y.last_hidden_state)